In [1]:
import pyzx as zx
from pyzx.basicrules import fuse
from fractions import Fraction
import numpy as np
import sympy as sp
from collections import defaultdict
import matplotlib.pyplot as plt
import sys

np.set_printoptions(threshold=np.inf) # print the whole matrix

In [2]:
master_0_phase = Fraction(1,3) # represents 0, ...
master_pi_phase = Fraction(2,3) # represents pi, ...
star_phase = Fraction(5,3) # represents triangle and NOT, ...
# ... which makes searching easier (TODO: I could have used vdata)

In [3]:
Z = zx.VertexType.Z
X = zx.VertexType.X
B = zx.VertexType.BOUNDARY
W_INP = zx.VertexType.W_INPUT
W_OUT = zx.VertexType.W_OUTPUT
SE = zx.EdgeType.SIMPLE
HE = zx.EdgeType.HADAMARD

In [4]:
draw_scale_default = 30

In [5]:
def nice_print(str, conditional=True):
    if conditional:
        print(str)

# like zx.draw but with my favorite settings already set
def nice_draw(g, conditional=True, labels=True, scale=draw_scale_default):
    if not conditional:
        return
    
    exact_string = str(g.scalar).split(" = ")[1]
    numeric_string = str(g.scalar.to_number())
    print("\t" + exact_string + " ≈ " + numeric_string)

    zx.draw(g, labels=labels, scale=scale)

In [6]:
def to_pure_matrix(matrix):
    # get rid of floating point error
    # change -0.0 to 0.0
    return np.around(matrix, 10) + 0.

In [7]:
# checks whether arr1 ≈ x*arr2 for integer x
def allclose_up_to_scalar(arr1, arr2, rtol=1e-05, atol=1e-08):
    if arr1.shape != arr2.shape:
        return False
    
    scalar = None
    for i in range(arr1.size):
        if arr1[i] != 0:
            if scalar is None:
                scalar = arr2[i] / arr1[i]
            elif not np.isclose(arr2[i], arr1[i] * scalar, rtol=rtol, atol=atol):
                return False
        elif arr2[i] != 0:
            return False
    
    return True

# converts all the diagrams in cliffs and non_cliffs list to statevector representation
# and then sums all of them up and compares result with orig_statevec
def check_decomp(qcirc, orig_statevec, cliffs, non_cliffs):
    final_statevec = np.zeros(2**qcirc.e_cnt, dtype=complex)

    for cliff in cliffs:
        cliff = cliff.copy()

        qcirc.revert_master_phases(cliff)
        zx.full_reduce(cliff)

        statevec = to_pure_matrix(cliff.to_matrix()).flatten()
        final_statevec += statevec
    
    for non_cliff in non_cliffs:
        non_cliff = non_cliff.copy()

        qcirc.revert_star_placeholder(non_cliff)
        qcirc.revert_master_phases(non_cliff)
        zx.full_reduce(non_cliff)

        statevec = to_pure_matrix(non_cliff.to_matrix()).flatten()
        final_statevec += statevec

    final_statevec = final_statevec / np.linalg.norm(final_statevec) # necessary, orig_statevec was also normalized

    print("orig_statevec:")
    print(orig_statevec)
    print("final_statevec:")
    print(final_statevec)
    print("-->")
    # sanity check
    sum = 0
    for element in final_statevec:
        sum += np.abs(element) ** 2
    print("sum:", sum)
    print("allclose_up_to_scalar:", allclose_up_to_scalar(orig_statevec, final_statevec))
    print("allclose:", np.allclose(orig_statevec, final_statevec))

    return np.allclose(orig_statevec, final_statevec)

In [8]:
# called by recursive decomp algorithm when discovering a disconected Z-Star-Z configuration, which
# is easy to convert to a scalar (that is automatically changed in g)
def convert_z_isolated_star_to_scalar(g, v1, star, v2):
    A = sp.Matrix([[1, 1], [1, 0]]) # star matrix

    def get_state_vec_components(v):
        if g.type(v) == Z and (g.phase(v) == 0 or g.phase(v) == master_0_phase):
            return sp.Matrix([1, 1])
        elif g.type(v) == Z and (g.phase(v) == 1 or g.phase(v) == master_pi_phase):
            return sp.Matrix([1, -1])
        else:
            print("INVALID SCALAR PASSED TO FUNCTION")
            sys.exit(0)
    
    x = get_state_vec_components(v1)
    y = get_state_vec_components(v2)

    Ay = A * y  # matrix-vector product A * y
    result = x.dot(Ay)  # inner product <x | (A * y)>
    
    g.remove_vertex(v1)
    g.remove_vertex(star)
    g.remove_vertex(v2)

    if result == 3:
        g.scalar.add_float(3)
    elif result == 1:
        pass
    elif result == -1:
        g.scalar.add_phase(Fraction(1,1))
    else:
        print("IMPOSSIBLE (?) SCALAR ACHIEVED")
        sys.exit(0)

# pushes as normal, but also fuses spiders of same color and is considerate
# of placeholders. returns same vertex if can't be pushed further, returns
# list of newly generated vertices otherwise
def generic_state_push(g, v, leave_scalar_unchanged=False, consider_had_edge=False):
    vtype = g.type(v)
    vphase = g.phase(v)

    if not (g.vertex_degree(v) == 1 and (vphase in [0,1, master_0_phase, master_pi_phase]) and (vtype in [Z,X])):
        return [v]

    nv = next(iter(g.neighbors(v)))
    nvtype = g.type(nv)

    def apply_color_change():
        if vtype == Z:
            g.set_type(v, X)
        elif vtype == X:
            g.set_type(v, Z)
        
        nv_nv = None
        for n in g.neighbors(nv):
            if n != v:
                nv_nv = n
                break

        nv_nv_nv = None
        for n in g.neighbors(nv_nv):
            if n != nv:
                nv_nv_nv = n
                break

        g.remove_vertex(nv)
        g.remove_vertex(nv_nv)
        g.add_edge((v, nv_nv_nv))

        return nv_nv_nv

    if consider_had_edge:
        if "placeholder" in g.vdata_keys(nv):
            if g.vdata(nv, "placeholder") == "had_edge_side_vertex":
                nv = apply_color_change()
                nvtype = g.type(nv)
                vtype = g.type(v)
                # print("AFTER")
                # print("nv:", nv)
                # print("nvtype:", nvtype)
                # nice_draw(g)

    if not nvtype in [Z,X]:
        return [v]
    
    if g.vertex_degree(nv) == 1: # scalar
        return [v]

    if g.phase(nv) == star_phase:
        neighbor_of_nv = list(g.neighbors(nv))
        neighbor_of_nv.remove(v)

        if len(neighbor_of_nv) > 1:
            print("DIAGRAM IS INVALID! CAN'T HAVE MULTIPLE EDGES CONNECTED TO A STAR SPIDER!")
            sys.exit(0)

        neighbor_of_nv = neighbor_of_nv[0]
        
        if g.type(v) != X:
            if g.vertex_degree(neighbor_of_nv) > 1:
                return [v]
            elif g.type(neighbor_of_nv) == X:
                return [neighbor_of_nv]
            else:
                orig_v = v
                convert_z_isolated_star_to_scalar(g, v, nv, neighbor_of_nv) # v gets removed
                return [orig_v]

        master_used = False
        if vphase == master_0_phase:
            g.set_phase(v, 0)
            vphase = 0
            master_used = True
        elif vphase == master_pi_phase:
            g.set_phase(v, Fraction(1,1))
            vphase = 1
            master_used = True

        if vphase == 0:
            g.scalar.add_power(1)
            g.set_type(v, Z)
        else:
            g.set_phase(v, 0)
        
        new_v = g.add_vertex(g.type(v), g.qubit(nv), g.row(nv), phase=g.phase(v))

        if master_used and vphase == 0:
            g.set_phase(new_v, master_0_phase)
        elif master_used and vphase == 1:
            g.set_phase(new_v, master_pi_phase)

        g.remove_vertex(v)
        g.remove_vertex(nv)
        g.add_edge((new_v, neighbor_of_nv))

        return [new_v]

    if vtype == nvtype:
        master_used = False
        if vphase == master_0_phase:
            g.set_phase(v, 0)
            master_used = True
        elif vphase == master_pi_phase:
            g.set_phase(v, Fraction(1,1))
            master_used = True
        if g.phase(nv) == master_0_phase:
            g.set_phase(nv, 0)
            master_used = True
        elif g.phase(nv) == master_pi_phase:
            g.set_phase(nv, Fraction(1,1))
            master_used = True

        fuse(g, nv, v)

        new_v = g.add_vertex(g.type(nv), g.qubit(nv), g.row(nv), phase=g.phase(nv))

        # rare 'bug', sometimes new_v same as v
        if new_v == v:
            orig_new_v = new_v
            new_v = g.add_vertex(g.type(nv), g.qubit(nv), g.row(nv), phase=g.phase(nv))
            g.remove_vertex(orig_new_v)

        if master_used and g.phase(nv) == 0:
            g.set_phase(new_v, master_0_phase)
        elif master_used and g.phase(nv) == 1:
            g.set_phase(new_v, master_pi_phase)

        neighbors_of_nv = list(g.neighbors(nv))
        for neighbor in neighbors_of_nv:
            g.add_edge((new_v, neighbor))
        g.remove_vertex(nv)

        if g.vertex_degree(new_v) == 1:
            return [new_v]
        
        return [v]
    
    new_v = []

    neighbors_of_nv = list(g.neighbors(nv))
    neighbors_of_nv.remove(v)

    # \frac{e^{i*a*alpha}}{\sqrt{2}^{n-1}}, v has phase a*pi, alpha is nvphase
    a = 1 if vphase in [1, master_pi_phase] else 0
    nvphase = g.phase(nv)
    if nvphase in [0, 1]:
        alpha = nvphase
    elif nvphase == master_0_phase:
        alpha = 0
    elif nvphase == master_pi_phase:
        alpha = Fraction(1,1)
    else:
        print("INVALID NEIGHBOR OF PUSHING TARGET")
        sys.exit(0)
    n = len(neighbors_of_nv)
    
    if not leave_scalar_unchanged:
        g.scalar.add_phase(a*alpha)
        g.scalar.add_power(-(n-1))

    for neighbor in neighbors_of_nv:
        new_v.append(g.add_vertex(vtype, g.qubit(nv), g.row(nv), phase=vphase))
        g.add_edge((new_v[-1], neighbor))
    g.remove_vertex(v)
    g.remove_vertex(nv)
    
    return new_v

In [9]:
class QCirc:
    # prepares init_subgraphs (IS) with states according to size of E, A and OUT register
    def __init__(self, e_cnt, a_cnt, out_cnt):
        # initially, there is only one graph, which is split into "subgraphs"
        self.init_subgraphs = []

        # afterwards, there are lots of diagrams - whilst not fully decomposed, they are in non_cliffs, once they are, they go into cliffs
        self.non_cliffs = []
        self.cliffs = []

        # can be set for debugging purposes (the statevector of the original circuit with effects applied)
        # should not be used for complex topologies since this is obviously very inefficient (otherwise no need for this whole algorithm)
        self.orig_statevec = None

        # can be set for debugging/benchmarking purposes
        self.t_repr_graph = None

        self.e_cnt = e_cnt
        self.a_cnt = a_cnt
        self.out_cnt = out_cnt

        cur = zx.Graph()

        for i in range(self.e_cnt):
            z = cur.add_vertex(Z, qubit=i, row=0)
            end = cur.add_vertex(B, qubit=i, row=1)
            cur.add_edge((z, end))

        for i in range(self.e_cnt, self.e_cnt+self.a_cnt):
            x = cur.add_vertex(X, qubit=i, row=0, phase=Fraction(1,1))
            end = cur.add_vertex(B, qubit=i, row=1)
            cur.add_edge((x, end))
        
        for i in range(self.e_cnt+self.a_cnt, self.e_cnt+self.a_cnt+self.out_cnt):
            z = cur.add_vertex(Z, qubit=i, row=0, phase=Fraction(1,1))
            end = cur.add_vertex(B, qubit=i, row=1)
            cur.add_edge((z, end))
        
        cur.auto_detect_io()
        self.init_subgraphs.append(cur)

    # can be set for debugging purposes (the statevector of the original circuit with effects applied)
    # should not be called for complex topologies since this is obviously very inefficient (otherwise no need for this whole algorithm)
    def set_orig_statevec(self):
        orig_qcirc = self.get_merged_IS().copy()
        self.apply_effect_scalar_invariant(orig_qcirc, "/"*self.e_cnt + "1"*self.a_cnt + "-"*self.out_cnt)
        self.revert_master_phases(orig_qcirc)

        orig_statevec = to_pure_matrix(orig_qcirc.to_matrix()).flatten()
        orig_statevec = orig_statevec / np.linalg.norm(orig_statevec)

        self.orig_statevec = orig_statevec

    ### Following section contain functions that work with the init_subgraphs (IS) representation ###

    # returns composition of all elements in IS into one graph
    def get_merged_IS(self):
        whole_graph = self.init_subgraphs[0]
        for i in range(1, len(self.init_subgraphs)):
            whole_graph = whole_graph + self.init_subgraphs[i]
        
        return whole_graph

    # replaces all triangles with equivalent diagram consisting of T gates
    # not used for this algorithm, but may be used when benchmarking against other algorithms, which require use of T-gates instead of triangles
    def get_t_gate_repr(self):
        g = self.get_merged_IS()

        vertices_to_be_deleted = []
        for v in list(g.vertices()):              
            if v in vertices_to_be_deleted:
                continue
            
            if g.type(v) == W_INP:
                neighbors = g.neighbors(v)

                # this assumes there will never be a triangle connected to something else than a NOT-gate (which is always the case for toffoli gates)
                w_out = None
                x_pi = None
                for neighbor in neighbors:
                    if g.type(neighbor) == W_OUT:
                        w_out = neighbor
                        continue
                    x_pi = neighbor
                
                w_out_neighbors = [neighbor for neighbor in g.neighbors(w_out) if neighbor != v]
                x_pi_neighbor = [neighbor for neighbor in g.neighbors(x_pi) if neighbor != v][0]

                # triangle has a state still attached, remove it
                w_out_neighbor = None
                for j in range(len(w_out_neighbors)):
                    if len(list(g.neighbors(w_out_neighbors[j]))) == 1:
                        vertices_to_be_deleted.append(w_out_neighbors[j])
                    else:
                        w_out_neighbor = w_out_neighbors[j]

                vertices_to_be_deleted.append(v)
                vertices_to_be_deleted.append(w_out)
                vertices_to_be_deleted.append(x_pi)

                new_qubit_pos = g.qubit(x_pi)
                new_row_pos = g.row(x_pi)

                x1 = g.add_vertex(X, qubit=new_qubit_pos, row=new_row_pos-0.5)
                z1 = g.add_vertex(Z, qubit=new_qubit_pos-0.5, row=new_row_pos, phase=Fraction(-1,4))
                z2 = g.add_vertex(Z, qubit=new_qubit_pos+0.5, row=new_row_pos, phase=Fraction(1,4))
                x2 = g.add_vertex(X, qubit=new_qubit_pos-0.5, row=new_row_pos+0.5)
                x3 = g.add_vertex(X, qubit=new_qubit_pos+0.5, row=new_row_pos+0.5)
                z3 = g.add_vertex(Z, qubit=new_qubit_pos-1, row=new_row_pos+0.5, phase=Fraction(-1,4))
                z4 = g.add_vertex(Z, qubit=new_qubit_pos+1, row=new_row_pos+0.5, phase=Fraction(1,4))
                z5 = g.add_vertex(Z, qubit=new_qubit_pos, row=new_row_pos+1)
                x_not = g.add_vertex(X, qubit=new_qubit_pos, row=new_row_pos+1.5, phase=Fraction(1,1))

                g.add_edge((x_pi_neighbor, x1))
                g.add_edge((x1, z1))
                g.add_edge((z1, x2))
                g.add_edge((x2, z3))
                g.add_edge((x2, z5))
                g.add_edge((x1, z2))
                g.add_edge((z2, x3))
                g.add_edge((x3, z4))
                g.add_edge((x3, z5))
                g.add_edge((z5, x_not))
                g.add_edge((x_not, w_out_neighbor))

                g.scalar.add_power(1)

        for v in vertices_to_be_deleted:
            g.remove_vertex(v)

        return g

    # store the T-gate representation graph for later use (if needed for benchmarking)
    def set_t_gate_repr_graph(self):
        self.t_repr_graph = self.get_t_gate_repr()

    # adds new element to IS, consisting of NOT gates at qubit positions specified
    # by idx_list, identities everywhere else
    def add_not_gates_IS(self, idx_list):
        cur = zx.Graph()

        for i in range(self.e_cnt+self.a_cnt+self.out_cnt):
            if i in idx_list:
                start = cur.add_vertex(B, qubit=i, row=0)
                not_gate = cur.add_vertex(X, qubit=i, row=1, phase=Fraction(1,1))
                end = cur.add_vertex(B, qubit=i, row=2)
                cur.add_edge((start, not_gate))
                cur.add_edge((not_gate, end))
            else:
                start = cur.add_vertex(B, qubit=i, row=0)
                end = cur.add_vertex(B, qubit=i, row=2)
                cur.add_edge((start, end))
        
        cur.auto_detect_io()
        self.init_subgraphs.append(cur)
    
    # adds new element to IS, namely a multi-controlled toffoli gate (control qubits specified by ctrl_idx_list, target qubit by targ_idx)
    # if pi_inside == False, then one of the two X Pi spiders making up the ZX-Representation will be pushed out onto main "qubit wires" 
    # assumes targ_idx > any(ctrl_idx_list) (aka below them)
    def add_toffoli_gates_IS(self, ctrl_idx_list, targ_idx, pi_inside=True):
        if not pi_inside:
            self.add_not_gates_IS(ctrl_idx_list)

        cur = zx.Graph()

        z_verts = [] # notice this will be sorted from smallest to biggest
        x_vert = None

        for i in range(self.e_cnt+self.a_cnt+self.out_cnt):
            if i in ctrl_idx_list:
                start = cur.add_vertex(B, qubit=i, row=0)
                z = cur.add_vertex(Z, qubit=i, row=1)
                z_verts.append(z)
                end = cur.add_vertex(B, qubit=i, row=3)
                cur.add_edge((start, z))
                cur.add_edge((z, end))
            elif i == targ_idx:
                start = cur.add_vertex(B, qubit=i, row=0)
                x = cur.add_vertex(X, qubit=i, row=1)
                x_vert = x
                end = cur.add_vertex(B, qubit=i, row=3)
                cur.add_edge((start, x))
                cur.add_edge((x, end))
            else:
                start = cur.add_vertex(B, qubit=i, row=0)
                end = cur.add_vertex(B, qubit=i, row=3)
                cur.add_edge((start, end))
        
        targ_shift = cur.add_vertex(Z, qubit=-0.5, row=3, phase=Fraction(1,1))
        not_shift = cur.add_vertex(X, qubit=-1, row=3, phase=Fraction(1,1))
        b_shift = cur.add_vertex(B, qubit=-1, row=3.5)
        cur.add_edge((x_vert, targ_shift))
        cur.add_edge((targ_shift, not_shift))
        cur.add_edge((not_shift, b_shift))

        for i, z_vert in enumerate(z_verts):
            if pi_inside:
                x_pi = cur.add_vertex(X, qubit=-len(z_verts)+i-1, row=2.5, phase=Fraction(1,1))
                not_shift = cur.add_vertex(X, qubit=-len(z_verts)+i-1, row=3, phase=Fraction(1,1))
                b_shift = cur.add_vertex(B, qubit=-len(z_verts)+i-1, row=3.5)
                cur.add_edge((z_vert, x_pi))
                cur.add_edge((x_pi, not_shift))
                cur.add_edge((not_shift, b_shift))
            else:
                not_shift = cur.add_vertex(X, qubit=-len(z_verts)+i-1, row=3, phase=Fraction(1,1))
                b_shift = cur.add_vertex(B, qubit=-len(z_verts)+i-1, row=3.5)
                cur.add_edge((z_vert, not_shift))
                cur.add_edge((not_shift, b_shift))

        triangle = zx.generate.spider("W",1,2) + (zx.generate.spider("Z", 1, 0) @ zx.generate.identity(1))
        # hardcoded positions for triangle (by trial-and-error)
        q_pos_list = [0, 0, 0, -0.25, 0]
        q_pos_list = [-(len(z_verts)+1)+x for x in q_pos_list]
        r_pos_list = [0.2, 0.35, 0, 0.7, 1]
        for i, v in enumerate(triangle.vertices()):
            triangle.set_position(v, q_pos_list[i], r_pos_list[i])
        triangles = triangle
        
        for i in range(len(z_verts)):
            triangle = zx.generate.spider("W",1,2) + (zx.generate.spider("Z", 1, 0) @ zx.generate.identity(1))
            # hardcoded positions for triangle (by trial-and-error)
            q_pos_list = [0, 0, 0, -0.25, 0]
            r_pos_list = [0.2, 0.35, 0, 0.7, 1]
            for i, v in enumerate(triangle.vertices()):
                triangle.set_position(v, q_pos_list[i], r_pos_list[i])
            
            triangles = triangles @ triangle
        
        for i in range(self.e_cnt+self.a_cnt+self.out_cnt):
            triangles = triangles @ zx.generate.identity(1)

        after_triangle = zx.Graph()
        mid_idx = len(ctrl_idx_list + [targ_idx])/2
        mid_idx = round(mid_idx * 2) / 2 # round to nearest 0.5
        if int(mid_idx) == mid_idx: # add 0.5 if ending in .0
            mid_idx += 0.5
        master = after_triangle.add_vertex(Z, qubit=-mid_idx, row=1, phase=master_pi_phase)
        final_not = after_triangle.add_vertex(X, qubit=-1, row=1, phase=Fraction(1,1))

        for i in range(len(z_verts)+1):
            bound = after_triangle.add_vertex(B, qubit=-(len(z_verts)+1)+i, row=0)
            if i == len(z_verts):
                after_triangle.add_edge((bound, final_not))
                after_triangle.add_edge((final_not, master))
            else:
                after_triangle.add_edge((bound, master))
        
        for i in range(self.e_cnt+self.a_cnt+self.out_cnt):
            after_triangle = after_triangle @ zx.generate.identity(1)
        
        cur.auto_detect_io()
        triangles.auto_detect_io()
        after_triangle.auto_detect_io()
        cur = cur + triangles + after_triangle
        cur.auto_detect_io()
        self.init_subgraphs.append(cur)

        if not pi_inside:
            self.add_not_gates_IS(ctrl_idx_list)

    # adds new element to IS, namely the Grover diffusion operator U_s=2|s><s|-I
    def apply_diffusion_operator_IS(self):
        left = zx.Graph()

        for i in range(self.e_cnt):
            if i != self.e_cnt - 1:
                start = left.add_vertex(B, qubit=i, row=0)
                z1 = left.add_vertex(Z, qubit=i, row=1)
                z2 = left.add_vertex(Z, qubit=i, row=2)
                x_pi = left.add_vertex(X, qubit=i, row=3, phase=Fraction(1,1))
                end = left.add_vertex(B, qubit=i, row=4)
                left.add_edge((start, z1))
                left.add_edge((z1, z2), HE)
                left.add_edge((z2, x_pi))
                left.add_edge((x_pi, end))

                left.set_vdata(z1, "placeholder", "had_edge_side_vertex")
                left.set_vdata(z2, "placeholder", "had_edge_side_vertex")
            else:
                start = left.add_vertex(B, qubit=i, row=0)
                z_pi = left.add_vertex(Z, qubit=i, row=2, phase=Fraction(1,1))
                end = left.add_vertex(B, qubit=i, row=3)
                left.add_edge((start, z_pi))
                left.add_edge((z_pi, end))
        
        for i in range(self.e_cnt, self.e_cnt + self.a_cnt + self.out_cnt):
            start = left.add_vertex(B, qubit=i, row=0)
            end = left.add_vertex(B, qubit=i, row=3)
            left.add_edge((start, end))

        # TODO: actually this doesn't change anything in the end
        # doesn't matter which of the subgraphs gets the new scalar, just one of them, since merged afterwards anyways
        left.scalar.add_power(2+(self.e_cnt-1))

        left.auto_detect_io()
        self.init_subgraphs.append(left)

        self.add_toffoli_gates_IS([i for i in range(self.e_cnt-1)], self.e_cnt-1)

        right = left.adjoint()
        self.init_subgraphs.append(right)

    # commutes a column of NOT gates to the left, if there are obstructions in the way, pushing may change toffoli structure
    # both x_pi and toffoli graph have to be in initial default form, don't use after further simplifications!
    # not sure about pi_inside=True version
    def x_pi_commute_to_left_IS(self, x_pi_subgraph_idx):
        if x_pi_subgraph_idx <= 1:
            return False
        
        not_qubits = []
        for v in self.init_subgraphs[x_pi_subgraph_idx].vertices():
            if self.init_subgraphs[x_pi_subgraph_idx].type(v) != X or self.init_subgraphs[x_pi_subgraph_idx].phase(v) != 1:
                continue

            not_qubits.append(self.init_subgraphs[x_pi_subgraph_idx].qubit(v))

        self.init_subgraphs[x_pi_subgraph_idx-1], self.init_subgraphs[x_pi_subgraph_idx] = self.init_subgraphs[x_pi_subgraph_idx], self.init_subgraphs[x_pi_subgraph_idx-1]
        
        for v in self.init_subgraphs[x_pi_subgraph_idx].copy().vertices():
            cur_qubit = self.init_subgraphs[x_pi_subgraph_idx].qubit(v)

            if not cur_qubit in not_qubits:
                continue
            
            cur_row = self.init_subgraphs[x_pi_subgraph_idx].row(v)
            if cur_row != 1:
                continue

            cur_type = self.init_subgraphs[x_pi_subgraph_idx].type(v)
            if cur_type != Z:
                continue
            
            neighbors = self.init_subgraphs[x_pi_subgraph_idx].neighbors(v)

            for neighbor in neighbors:
                if self.init_subgraphs[x_pi_subgraph_idx].type(neighbor) == B:
                    continue

                if self.init_subgraphs[x_pi_subgraph_idx].type(neighbor) == X and self.init_subgraphs[x_pi_subgraph_idx].phase(neighbor) == 1:
                    neighbor_neighbors = self.init_subgraphs[x_pi_subgraph_idx].neighbors(neighbor)

                    # two NOT gates
                    if self.init_subgraphs[x_pi_subgraph_idx].type(list(neighbor_neighbors)[1]) == X and self.init_subgraphs[x_pi_subgraph_idx].phase(list(neighbor_neighbors)[1]) == 1:
                        self.init_subgraphs[x_pi_subgraph_idx].add_edge((v, list(neighbor_neighbors)[1]))

                        self.init_subgraphs[x_pi_subgraph_idx].remove_vertex(neighbor)
                    # one NOT gate
                    else:
                        new = self.init_subgraphs[x_pi_subgraph_idx].add_vertex(X, qubit=self.init_subgraphs[x_pi_subgraph_idx].qubit(neighbor), row=2.5, phase=Fraction(1,1))
                        self.init_subgraphs[x_pi_subgraph_idx].add_edge((v, new))
                        
                        self.init_subgraphs[x_pi_subgraph_idx].add_edge((new, neighbor))

                        self.init_subgraphs[x_pi_subgraph_idx].remove_edge(list(self.init_subgraphs[x_pi_subgraph_idx].incident_edges(neighbor))[1])
                    break
            
        return True
    
    # same as x_pi_commute_to_left_IS just to the right
    def x_pi_commute_to_right_IS(self, x_pi_subgraph_idx):
        if x_pi_subgraph_idx >= len(self.init_subgraphs)-1:
            return False
        
        not_qubits = []
        for v in self.init_subgraphs[x_pi_subgraph_idx].vertices():
            if self.init_subgraphs[x_pi_subgraph_idx].type(v) != X or self.init_subgraphs[x_pi_subgraph_idx].phase(v) != 1:
                continue

            not_qubits.append(self.init_subgraphs[x_pi_subgraph_idx].qubit(v))

        self.init_subgraphs[x_pi_subgraph_idx], self.init_subgraphs[x_pi_subgraph_idx+1] = self.init_subgraphs[x_pi_subgraph_idx+1], self.init_subgraphs[x_pi_subgraph_idx]
        
        for v in self.init_subgraphs[x_pi_subgraph_idx].copy().vertices():
            cur_qubit = self.init_subgraphs[x_pi_subgraph_idx].qubit(v)

            if not cur_qubit in not_qubits:
                continue
            
            cur_row = self.init_subgraphs[x_pi_subgraph_idx].row(v)
            if cur_row != 1:
                continue

            cur_type = self.init_subgraphs[x_pi_subgraph_idx].type(v)
            if cur_type != Z:
                continue
            
            neighbors = self.init_subgraphs[x_pi_subgraph_idx].neighbors(v)

            for neighbor in neighbors:
                if self.init_subgraphs[x_pi_subgraph_idx].type(neighbor) == B:
                    continue

                if self.init_subgraphs[x_pi_subgraph_idx].type(neighbor) == X and self.init_subgraphs[x_pi_subgraph_idx].phase(neighbor) == 1:
                    neighbor_neighbors = self.init_subgraphs[x_pi_subgraph_idx].neighbors(neighbor)

                    # two NOT gates
                    if self.init_subgraphs[x_pi_subgraph_idx].type(list(neighbor_neighbors)[1]) == X and self.init_subgraphs[x_pi_subgraph_idx].phase(list(neighbor_neighbors)[1]) == 1:
                        self.init_subgraphs[x_pi_subgraph_idx].add_edge((v, list(neighbor_neighbors)[1]))

                        self.init_subgraphs[x_pi_subgraph_idx].remove_vertex(neighbor)
                    # one NOT gate
                    else:
                        new = self.init_subgraphs[x_pi_subgraph_idx].add_vertex(X, qubit=self.init_subgraphs[x_pi_subgraph_idx].qubit(neighbor), row=2.5, phase=Fraction(1,1))
                        self.init_subgraphs[x_pi_subgraph_idx].add_edge((v, new))
                        
                        self.init_subgraphs[x_pi_subgraph_idx].add_edge((new, neighbor))

                        self.init_subgraphs[x_pi_subgraph_idx].remove_edge(list(self.init_subgraphs[x_pi_subgraph_idx].incident_edges(neighbor))[1])
                    break
        
        return True
    
    # applies both x_pi_commute_to_left_IS and x_pi_commute_to_right_IS on the passed lists of qubit indices
    # cnot_pos lists have to be orderd as they appear in circuit from left to right
    def x_pi_commute_to_borders_IS(self, left_cnot_pos, right_cnot_pos):
        left_cnot_pos.sort()
        right_cnot_pos.sort(reverse=True)

        for pos in left_cnot_pos:
            status = True
            while status:
                status = self.x_pi_commute_to_left_IS(pos)
                pos -= 1
        
        for pos in right_cnot_pos:
            status = True
            while status:
                status = self.x_pi_commute_to_right_IS(pos)
                pos += 1
    
    # apply middle toffoli decomposition -> starts with initial_subgraphs, at end creates two non_cliffs
    def split_mid_toffoli_IS(self, mid_toffoli_pos, mid_toffoli_ctrls, mid_toffoli_main_ctrl):
        first_half_diagram = self.init_subgraphs[0]
        for i in range(1, mid_toffoli_pos-1):
            first_half_diagram = first_half_diagram + self.init_subgraphs[i]
        
        second_half_diagram = self.init_subgraphs[mid_toffoli_pos+2]
        for i in range(mid_toffoli_pos+3, len(self.init_subgraphs)):
            second_half_diagram = second_half_diagram + self.init_subgraphs[i]
        
        self.init_subgraphs = [] # we don't need it anymore

        # first decomposition summand
        self.non_cliffs.append(first_half_diagram + second_half_diagram)

        self.non_cliffs[0].scalar.add_power(-1)

        # second decomposition summand
        mid_graph = zx.Graph()
        
        for i in range(self.a_cnt + self.e_cnt + self.out_cnt):
            if i in mid_toffoli_ctrls:
                b1 = mid_graph.add_vertex(B, qubit=i, row=0)
                x1 = mid_graph.add_vertex(X, qubit=i, row=1, phase=Fraction(1,1))
                x2 = mid_graph.add_vertex(X, qubit=i, row=2, phase=Fraction(1,1))
                b2 = mid_graph.add_vertex(B, qubit=i, row=3)

                mid_graph.add_edge((b1, x1))
                mid_graph.add_edge((x2, b2))
            elif i == mid_toffoli_main_ctrl:
                b1 = mid_graph.add_vertex(B, qubit=i, row=0)
                z1 = mid_graph.add_vertex(Z, qubit=i, row=1, phase=Fraction(1,1))
                z2 = mid_graph.add_vertex(Z, qubit=i, row=2, phase=Fraction(1,1))
                b2 = mid_graph.add_vertex(B, qubit=i, row=3)

                mid_graph.add_edge((b1, z1))
                mid_graph.add_edge((z2, b2))
            else:
                b1 = mid_graph.add_vertex(B, qubit=i, row=0)
                b2 = mid_graph.add_vertex(B, qubit=i, row=3)

                mid_graph.add_edge((b1, b2))

        mid_graph.auto_detect_io()
        self.non_cliffs.append(first_half_diagram + mid_graph + second_half_diagram)

        self.non_cliffs[1].scalar.add_phase(Fraction(1,1))
        m = len(mid_toffoli_ctrls)
        self.non_cliffs[1].scalar.add_power(-(1+2*m))

    ### Following section doesn't work with initial_subgraphs (IS) representation anymore ###

    # adds the Grover diffusion operator U_s=2|s><s|-I on the right of the given diagram
    # this implementation is a bit hacky since we're using the IS infrastructure whilst temporarily saving the IS state
    def apply_diffusion_operator_complete_graph(self, g):
        init_subgraphs_orig = self.init_subgraphs.copy()
        self.init_subgraphs = []

        self.apply_diffusion_operator_IS()
        right_side = self.get_merged_IS()

        self.init_subgraphs = init_subgraphs_orig

        return g + right_side
        
    # stack1 = two NOT gates | stack2 = one NOT gate
    # NOTE: this function can only be called right after creating the two summands
    # because it assumes if graph_idx == 0 then there is identity in the middle
    # and if graph_idx == 1 then the 0th qubit wire has decomp in it, thus does not create
    # the stack on that line
    # NOTE: cur_qubit_as_idx not so nice (but working) implementation, it needs to be reduced when graph_idx == 1
    # because then e.g. stack1_nodes[0] is at position 1, so we can't just do stack1_nodes[cur_qubit]
    def add_stacks(self, graph_idx):
        left_row_bound = 0
        right_row_bound = 0

        # search for first toffoli ctrl gate from left side on, then stop (so actually only a few iterations)
        for v in self.non_cliffs[graph_idx].vertices():
            if self.non_cliffs[graph_idx].type(v) == Z:
                if len(list(self.non_cliffs[graph_idx].incident_edges(v))) >= 3:
                    left_row_bound = self.non_cliffs[graph_idx].row(v)
                    break
        
        # search for last green spider (master node from last toffoli gate) from right side on, then stop 
        for v in range(len(self.non_cliffs[graph_idx].vertices())-1, -1, -1):
            if self.non_cliffs[graph_idx].type(v) == Z and self.non_cliffs[graph_idx].phase(v) == 1:
                right_row_bound = self.non_cliffs[graph_idx].row(v)
                break

        print("left_row_bound:", left_row_bound)
        print("right_row_bound:", right_row_bound)

        stack1_row = np.floor(left_row_bound + (right_row_bound-left_row_bound)/3)
        stack2_row = np.ceil(left_row_bound + 2*(right_row_bound-left_row_bound)/3)

        left_bounds = []
        right_bounds = []
        z_nodes = []

        for v in self.non_cliffs[graph_idx].vertices():
            if self.non_cliffs[graph_idx].qubit(v) < 0 or self.non_cliffs[graph_idx].qubit(v) > self.e_cnt - 1:
                continue

            if graph_idx == 1 and self.non_cliffs[graph_idx].qubit(v) == 0:
                continue

            if self.non_cliffs[graph_idx].row(v) >= left_row_bound and self.non_cliffs[graph_idx].row(v) <= right_row_bound:
                if self.non_cliffs[graph_idx].type(v) == Z: # technically useless since all are Z, just for sanity
                    z_nodes.append(v)

                continue

            if self.non_cliffs[graph_idx].row(v) + 1 == left_row_bound:
                left_bounds.append([self.non_cliffs[graph_idx].qubit(v), v])
                continue

            if self.non_cliffs[graph_idx].row(v) - 1 == right_row_bound:
                right_bounds.append([self.non_cliffs[graph_idx].qubit(v), v])
                continue

            neighbors = self.non_cliffs[graph_idx].neighbors(v)

            for neighbor in neighbors:
                if self.non_cliffs[graph_idx].row(neighbor) >= left_row_bound and self.non_cliffs[graph_idx].row(neighbor) <= right_row_bound:
                    if self.non_cliffs[graph_idx].row(v) < left_row_bound:
                        left_bounds.append([self.non_cliffs[graph_idx].qubit(v), v])
                    elif self.non_cliffs[graph_idx].row(v) > right_row_bound:
                        right_bounds.append([self.non_cliffs[graph_idx].qubit(v), v])
                    break
                # rare case where nothing is in between the bounds
                elif self.non_cliffs[graph_idx].row(neighbor) >= right_row_bound and self.non_cliffs[graph_idx].row(v) < left_row_bound:
                    left_bounds.append([self.non_cliffs[graph_idx].qubit(v), v])
                # rare case where nothing is in between the bounds
                elif self.non_cliffs[graph_idx].row(neighbor) <= left_row_bound and self.non_cliffs[graph_idx].row(v) > right_row_bound:
                    right_bounds.append([self.non_cliffs[graph_idx].qubit(v), v])

        left_bounds.sort(key=lambda x: x[0])
        left_bounds = [x[1] for x in left_bounds]

        right_bounds.sort(key=lambda x: x[0])
        right_bounds = [x[1] for x in right_bounds]
        
        print("left_bounds:", left_bounds)
        print("right_bounds:", right_bounds)
        print("z_nodes:", z_nodes)
        
        # zx.draw(self.non_cliffs[graph_idx], labels=True, scale=30, show_scalar=True)

        stack1_nodes = []
        stack2_nodes = []

        for i in range(self.e_cnt):
            if graph_idx == 1 and i == 0:
                continue

            cur_stack1_node = self.non_cliffs[graph_idx].add_vertex(Z, qubit=i, row=stack1_row, phase=master_0_phase)
            cur_stack2_node = self.non_cliffs[graph_idx].add_vertex(Z, qubit=i, row=stack2_row, phase=master_0_phase)

            stack1_nodes.append(cur_stack1_node)
            stack2_nodes.append(cur_stack2_node)

            bound_idx = i
            if graph_idx == 1:
                bound_idx -= 1
            
            self.non_cliffs[graph_idx].add_edge((left_bounds[bound_idx], cur_stack1_node))
            self.non_cliffs[graph_idx].add_edge((cur_stack1_node, cur_stack2_node))
            self.non_cliffs[graph_idx].add_edge((cur_stack2_node, right_bounds[bound_idx]))

        # edge case: no Z spider at all on one qubit -> need to manually remove edge
        special_case_qubits = []
        qubit_coverage_list = []
        for z_node in z_nodes:
            qubit_coverage_list.append(self.non_cliffs[graph_idx].qubit(z_node))
        qubit_coverage_list = list(dict.fromkeys(qubit_coverage_list)) # remove duplicates
        # print("qubit_coverage_list:", qubit_coverage_list)
        for i in range(self.e_cnt):
            if graph_idx == 1 and i == 0:
                continue
            
            if not i in qubit_coverage_list:
                special_case_qubits.append(i)

        # print("special_case_qubits:", special_case_qubits)
        for s in special_case_qubits:
            if graph_idx == 1:
                s -= 1

            self.non_cliffs[graph_idx].remove_edge(self.non_cliffs[graph_idx].edge(left_bounds[s], right_bounds[s]))

        not_gate_needed_qubits = []

        for z_node in z_nodes:
            cur_qubit = self.non_cliffs[graph_idx].qubit(z_node)

            cur_qubit_as_idx = cur_qubit
            if graph_idx == 1:
                    cur_qubit_as_idx -= 1

            first_neighbors = self.non_cliffs[graph_idx].neighbors(z_node)
            first_neighbor = None

            for cur_neighbor in first_neighbors:
                if self.non_cliffs[graph_idx].row(cur_neighbor) < left_row_bound or self.non_cliffs[graph_idx].row(cur_neighbor) > right_row_bound:
                    continue

                if self.non_cliffs[graph_idx].type(cur_neighbor) == X and self.non_cliffs[graph_idx].phase(cur_neighbor) == 1:
                    first_neighbor = cur_neighbor
                    break
            
            second_neighbors = self.non_cliffs[graph_idx].neighbors(first_neighbor)
            second_neighbor = None

            for cur_neighbor in second_neighbors:
                if self.non_cliffs[graph_idx].type(cur_neighbor) == X and self.non_cliffs[graph_idx].phase(cur_neighbor) == 1:
                    second_neighbor = cur_neighbor
                    break

            # one NOT gate
            if second_neighbor == None:
                self.non_cliffs[graph_idx].remove_vertex(z_node)
                self.non_cliffs[graph_idx].add_edge((first_neighbor, stack2_nodes[cur_qubit_as_idx]))
            # two NOT gates
            else:
                not_gate_needed_qubits.append(cur_qubit)
                self.non_cliffs[graph_idx].remove_vertex(z_node)
                self.non_cliffs[graph_idx].remove_vertex(first_neighbor)
                self.non_cliffs[graph_idx].add_edge((second_neighbor, stack1_nodes[cur_qubit_as_idx]))
        
        not_gate_needed_qubits = list(dict.fromkeys(not_gate_needed_qubits)) # remove duplicates
        not_gate_needed_qubits.sort()

        for cur_qubit in not_gate_needed_qubits:
            cur_qubit_as_idx = cur_qubit
            if graph_idx == 1:
                    cur_qubit_as_idx -= 1

            pos1 = np.round((left_row_bound + stack1_row)/2)
            pos2 = np.round((stack1_row + stack2_row)/2)

            node1 = self.non_cliffs[graph_idx].add_vertex(X, qubit=cur_qubit, row=pos1, phase=Fraction(1,1))
            node2 = self.non_cliffs[graph_idx].add_vertex(X, qubit=cur_qubit, row=pos2, phase=Fraction(1,1))

            self.non_cliffs[graph_idx].add_edge((left_bounds[cur_qubit_as_idx], node1))
            self.non_cliffs[graph_idx].add_edge((node1, stack1_nodes[cur_qubit_as_idx]))

            self.non_cliffs[graph_idx].add_edge((stack1_nodes[cur_qubit_as_idx], node2))
            self.non_cliffs[graph_idx].add_edge((node2, stack2_nodes[cur_qubit_as_idx]))

            self.non_cliffs[graph_idx].remove_edge(self.non_cliffs[graph_idx].edge(left_bounds[cur_qubit_as_idx], stack1_nodes[cur_qubit_as_idx]))
            self.non_cliffs[graph_idx].remove_edge(self.non_cliffs[graph_idx].edge(stack1_nodes[cur_qubit_as_idx], stack2_nodes[cur_qubit_as_idx]))

    # iterate through all non_cliffs and replace the star structure (aka triangle and NOT) with a Z spider with placeholder phase
    def substitute_star_placeholder(self):
        for i in range(len(self.non_cliffs)):

            vertices_to_be_deleted = []
            for v in list(self.non_cliffs[i].vertices()):              
                if v in vertices_to_be_deleted:
                    continue
                
                if self.non_cliffs[i].type(v) == W_INP:
                    neighbors = self.non_cliffs[i].neighbors(v)

                    w_out = None
                    x_pi = None
                    for neighbor in neighbors:
                        if self.non_cliffs[i].type(neighbor) == W_OUT:
                            w_out = neighbor
                            continue
                        x_pi = neighbor
                    
                    w_out_neighbors = [neighbor for neighbor in self.non_cliffs[i].neighbors(w_out) if neighbor != v]
                    x_pi_neighbor = [neighbor for neighbor in self.non_cliffs[i].neighbors(x_pi) if neighbor != v][0]

                    # triangle has a state still attached, remove it
                    w_out_neighbor = None
                    for j in range(len(w_out_neighbors)):
                        if len(list(self.non_cliffs[i].neighbors(w_out_neighbors[j]))) == 1:
                            vertices_to_be_deleted.append(w_out_neighbors[j])
                        else:
                            w_out_neighbor = w_out_neighbors[j]

                    vertices_to_be_deleted.append(v)
                    vertices_to_be_deleted.append(w_out)
                    vertices_to_be_deleted.append(x_pi)

                    new_qubit_pos = self.non_cliffs[i].qubit(x_pi)
                    new_row_pos = self.non_cliffs[i].row(x_pi)

                    new_z = self.non_cliffs[i].add_vertex(Z, qubit=new_qubit_pos, row=new_row_pos, phase=star_phase)

                    self.non_cliffs[i].add_edge((w_out_neighbor, new_z))
                    self.non_cliffs[i].add_edge((new_z, x_pi_neighbor))

            for v in vertices_to_be_deleted:
                self.non_cliffs[i].remove_vertex(v)

    # iterature through given graph and replace placeholders with actual star structure (aka triangle and NOT)
    def revert_star_placeholder(self, g):
        g_copy = g.copy()
        for v in g_copy.vertices():
            if g_copy.phase(v) == star_phase:
                neighbors = list(g_copy.neighbors(v))
                qubit_pos = g_copy.qubit(v)
                row_pos = g_copy.row(v)

                g.remove_vertex(v)

                x_pi = g.add_vertex(X, qubit=qubit_pos, row=row_pos, phase=Fraction(1,1))
                w_inp = g.add_vertex(W_INP, qubit=qubit_pos, row=row_pos+0.4)
                w_out = g.add_vertex(W_OUT, qubit=qubit_pos, row=row_pos+0.4+0.15)
                z = g.add_vertex(Z, qubit=qubit_pos-0.25, row=row_pos+0.4+0.5)
                
                g.add_edge((x_pi, w_inp))
                g.add_edge((w_inp, w_out))
                g.add_edge((w_out, z))

                g.add_edge((neighbors[0], x_pi))
                g.add_edge((neighbors[1], w_out))

    # reverts all placeholder phases that still might be present in cliffs diagrams, namely
    # phases of master nodes
    def revert_master_phases(self, g):
        for v in g.vertices():
            if g.phase(v) == master_0_phase:
                g.set_phase(v, 0)
            elif g.phase(v) == master_pi_phase:
                g.set_phase(v, Fraction(1,1))

    # iterate through given graph and check if there are any spiders with star_phase, if yes then returns True
    def is_cliff(self, graph):
        for v in graph.vertices():
            if graph.phase(v) == star_phase:
                return False
        
        return True
    
    # pushes all states found in graph, only that it does not consider states on the left-side (although
    # with passed exceptions)
    # it does consider the right side to potentially get some nice cancelling
    def left_side_restricted_complete_state_push(self, g, exception_qubit_start=1, exception_qubit_end=0, consider_had_edge=False):
        states = []

        for v in g.vertices():
            if g.vertex_degree(v) == 1:
                if g.row(v) != 0:
                    states.append(v)
                elif g.qubit(v) >= exception_qubit_start and g.qubit(v) < exception_qubit_end:
                    states.append(v)

        while len(states) > 0:
            if not states[0] in list(g.vertices()):
                continue
            
            new_states = generic_state_push(g, states[0], consider_had_edge=consider_had_edge)

            if new_states != [states[0]]:
                states.extend(new_states)
            
            states.pop(0)
    
    # pushes one or multiple(!) vertex until not possible anymore
    def recursive_state_push(self, g, v, leave_scalar_unchanged=False, consider_had_edge=False):
        if type(v) == int:
            states = [v]
        else:
            states = v

        while len(states) > 0:
            new_states = generic_state_push(g, states[0], leave_scalar_unchanged=leave_scalar_unchanged, consider_had_edge=consider_had_edge)

            if new_states != [states[0]]:
                states.extend(new_states)
            
            states.pop(0)

    # return list of all vertices that are Z spiders with master phase placeholders
    def get_all_pot_master_v(self, g):
        v_list = []
        for v in g.vertices():
            if g.type(v) == Z and g.phase(v) in [master_0_phase, master_pi_phase]:
                v_list.append(v)
        
        return v_list

    # uses get_all_pot_master_v to obtain potential master vertices, however
    # tries to group vertices with the same qubit index (only for vertices in E register, since
    # there the stacks are located)
    def get_pot_master_groups(self, g):
        v_list = self.get_all_pot_master_v(g)

        group_list = []

        # the negative qubits are not supposed to be grouped together
        cnt = 0
        for v in v_list:
            qbit = g.qubit(v)
            if qbit < 0:
                qbit -= cnt
                cnt += 1
            group_list.append([qbit, v])

        # AI-generated, see weighting_test.ipynb for what this doe
        grouped = {}
        for sublist in group_list:
            key, value = sublist
            if key not in grouped:
                grouped[key] = []
            grouped[key].append(value)
        intermediate = []
        for key, values in grouped.items():
            intermediate.append([key, values])
        final_result = []
        for item in intermediate:
            if isinstance(item[1], list):
                final_result.append(item[1])
            else:
                final_result.append([item[1]])

        return final_result   

    # calculates the weight of a given potential master node by searching "depth 2" (see documentation for more details)
    # might be called multiple times and then the weight is summed for one group
    def get_master_v_weight(self, g, v):
        extra_weight = 2

        total_weight = 0

        for l1_nv in g.neighbors(v):
            if g.phase(l1_nv) == star_phase:
                total_weight += 1
            elif not g.type(l1_nv) in [Z,X]:
                continue
            elif g.vertex_degree(l1_nv) > 2:
                continue
            else:
                continue

            l2_nv = list(g.neighbors(l1_nv))
            l2_nv.remove(v)

            if len(l2_nv) > 1:
                continue
            l2_nv = l2_nv[0]

            if g.type(l2_nv) != Z:
                continue

            l3_nv = list(g.neighbors(l2_nv))
            l3_nv.remove(l1_nv)

            for cur_v in l3_nv:
                if g.phase(cur_v) == star_phase:
                    total_weight += 1
                elif not g.type(cur_v) in [Z,X]:
                    continue
                elif g.vertex_degree(cur_v) > 2:
                    continue
                else:
                    extra_weight_valid = False
                    if g.type(cur_v) == X and g.phase(cur_v) == 1:
                        extra_weight_valid = True

                    cur_v = list(g.neighbors(cur_v))
                    cur_v.remove(l2_nv)
                    
                    if len(cur_v) > 1:
                        continue
                    cur_v = cur_v[0]
                    
                    if g.phase(cur_v) != star_phase:
                        continue
                    total_weight += extra_weight if extra_weight_valid else 1

        return total_weight

    # decompose passed master vertex into two diagrams that are checked using is_cliff and passed
    # into appropriate list (either cliffs or non_cliffs)
    def decomp_single_master(self, g, v, consider_had_edge=False):
        g1 = g.clone()
        g2 = g.clone()

        alpha = g.phase(v)
        n = 0
        m = 0

        g2_star_side_states = []

        neighbors = list(g.neighbors(v))
        for neighbor in neighbors:
            if g.phase(neighbor) != star_phase:
                n += 1
                continue
            
            m += 1
            star_neighbor = list(g.neighbors(neighbor))
            star_neighbor.remove(v)
            if len(star_neighbor) > 1:
                print("SPIDER AFTER STAR OF MASTER SPIDER BRANCH INVALID")
                sys.exit(0)
            star_neighbor = star_neighbor[0]

            # we don't need to do the following for g2, since it would get removed anyways when pushing the X state attached through it
            if g.type(star_neighbor) != Z:
                g1_z_replace = g1.add_vertex(Z, qubit=g.qubit(neighbor), row=g.row(neighbor))
                g1.add_edge((g1_z_replace, star_neighbor))

            g1.remove_vertex(neighbor)
            g2.remove_vertex(neighbor)

            g2_star_side_states.append(g2.add_vertex(X, qubit=g.qubit(star_neighbor), row=g.row(star_neighbor), phase=0))
            g2.add_edge((star_neighbor, g2_star_side_states[-1]))
        
        if n == 0:
            g1.remove_vertex(v)
            g2.remove_vertex(v)

        if n != 0:
            g1_non_star_side_state = g1.add_vertex(X, qubit=g.qubit(v), row=g.row(v), phase=0)
            g1.add_edge((v, g1_non_star_side_state))

            g2_non_star_side_state = g2.add_vertex(X, qubit=g.qubit(v), row=g.row(v), phase=Fraction(1,1))
            g2.add_edge((v, g2_non_star_side_state))

        g1.scalar.add_power(-n)

        if alpha == master_0_phase:
            # \frac{e^{i*0}}{\sqrt{2}^{n+m}}
            g2.scalar.add_power(-(n+m))
        elif alpha == master_pi_phase:
            # \frac{e^{i*pi}}{\sqrt{2}^{n+m}}
            g2.scalar.add_phase(Fraction(1,1))
            g2.scalar.add_power(-(n+m))
        else:
            print("MASTER NODE CAN'T HAVE INVALID PHASE")
            sys.exit(0)

        if n != 0:
            g1_returned_states = generic_state_push(g1, g1_non_star_side_state, leave_scalar_unchanged=True, consider_had_edge=consider_had_edge)
            g2_returned_states = generic_state_push(g2, g2_non_star_side_state, leave_scalar_unchanged=True, consider_had_edge=consider_had_edge)

        if n != 0:
            self.recursive_state_push(g1, g1_returned_states, leave_scalar_unchanged=False, consider_had_edge=consider_had_edge)
            self.recursive_state_push(g2, g2_returned_states, leave_scalar_unchanged=False, consider_had_edge=consider_had_edge)

        # if m is zero, then this loop won't iteratre through anything, so no condition needed
        for state in g2_star_side_states:
            self.recursive_state_push(g2, state, consider_had_edge=consider_had_edge)
        
        self.non_cliffs.remove(g)
        
        if self.is_cliff(g1):
            self.cliffs.append(g1)
        else:
            self.non_cliffs.append(g1)
        
        if self.is_cliff(g2):
            self.cliffs.append(g2)
        else:
            self.non_cliffs.append(g2)

    # decomposes a pair of master nodes (from E register) into two summands by simply
    # calling decomp_single_master and letting the recursive state pushing do it's job
    # TODO: of course less efficient than just implementing the precalculated decomposition
    # (see documentation for how this decomposition would look like)
    def decomp_pair_master(self, g, v1, v2):
        self.decomp_single_master(g, v1)

    # applies effect without changing the scalar - useful for debugging
    # it doesn't hurt since applied on all the same way and the statevector is normalized in the end again
    def apply_effect_scalar_invariant(self, g, effect_string):
        scalar_orig = g.scalar.copy()
        g.apply_effect(effect_string)

        g.scalar = scalar_orig

# Convert graph to graphviz-friendly string representation

In [10]:
def to_dot(g) -> str:
    dot = "graph {\n"
    
    for v in g.vertices():
        t = g.type(v)
        p = g.phase(v)
        
        color = {
            B: "black",
            Z: "green",
            X: "red"
        }.get(t, "black")
        
        label = ""
        if v in g.inputs():
            label = f"{v}:i"
        elif v in g.outputs():
            label = f"{v}:o"
        elif p != 0:
            label = f"{v}:{p}"
        else:
            label = f"{v}"
        
        dot += f"  {v} [color={color}, label=\"{label}\""
        
        q = g.qubit(v)
        r = g.row(v)
        if q != 0 or r != 0:
            dot += f", pos=\"{q},{r}!\""
        
        dot += "]\n"
    
    dot += "\n"
    
    for src, trg in g.edges():
        e = g.edge(src, trg)
        et = g.edge_type(e)

        dot += f"  {src} -- {trg}"
        if et == HE:
            dot += " [color=blue]"
        # if ty == EType.T:
        #     dot += " [color=orange]"
        dot += "\n"
    
    dot += "}\n"
    
    return dot

# Circuit creation

In [11]:
# Nomenclature taken from paper
E_CNT = 6
A_CNT = 3
OUT_CNT = 1 # always has to be zero but just to make it be clean

left_cnot_pos = []
right_cnot_pos = []
mid_toffoli_pos = 0

print("------------------------------------ Initial circuit creation ------------------------------------")

qcirc = QCirc(E_CNT, A_CNT, OUT_CNT)
qcirc.add_toffoli_gates_IS([0,1,4,5], 6)
qcirc.add_toffoli_gates_IS([2,3,4,5], 8)
qcirc.add_not_gates_IS([2,3,4,5])
left_cnot_pos.append(len(qcirc.init_subgraphs)-1)
qcirc.add_toffoli_gates_IS([0,1,2,3], 7)
qcirc.add_toffoli_gates_IS([2,3,4,5], 8)

mid_toffoli_ctrls = [0,6,7,8]
mid_toffoli_main_ctrl = 9
qcirc.add_toffoli_gates_IS(mid_toffoli_ctrls, mid_toffoli_main_ctrl, False)
mid_toffoli_pos = len(qcirc.init_subgraphs)-2

qcirc.add_toffoli_gates_IS([2,3,4,5], 8)
qcirc.add_toffoli_gates_IS([0,1,2,3], 7)
qcirc.add_not_gates_IS([2,3,4,5])
right_cnot_pos.append(len(qcirc.init_subgraphs)-1)
qcirc.add_toffoli_gates_IS([2,3,4,5], 8)
qcirc.add_toffoli_gates_IS([0,1,4,5], 6)

qcirc.apply_diffusion_operator_IS()

qcirc.non_cliffs = [qcirc.get_merged_IS()]
qcirc.substitute_star_placeholder()
qcirc.revert_master_phases(qcirc.non_cliffs[0])

# somehow quizx doesn't like 5pi/3 phase, but 1pi/3 is fine
for v in qcirc.non_cliffs[0].vertices():
    if qcirc.non_cliffs[0].phase(v) == star_phase:
        qcirc.non_cliffs[0].set_phase(v, Fraction(1, 3))

qcirc.apply_effect_scalar_invariant(qcirc.non_cliffs[0], "/"*E_CNT + "1"*A_CNT + "-"*OUT_CNT)

nice_draw(qcirc.non_cliffs[0])

dot_string = to_dot(qcirc.non_cliffs[0]).rstrip()
print(dot_string)
with open("python_generated_dot_2eloop.txt", "w") as file:
    file.write(dot_string)

------------------------------------ Initial circuit creation ------------------------------------
	sqrt(2)^14 ≈ (128.0000000000001+0j)


graph {
  0 [color=green, label="0"]
  1 [color=green, label="1", pos="1,0!"]
  2 [color=green, label="2", pos="2,0!"]
  3 [color=green, label="3", pos="3,0!"]
  4 [color=green, label="4", pos="4,0!"]
  5 [color=green, label="5", pos="5,0!"]
  6 [color=red, label="6:1", pos="6,0!"]
  7 [color=red, label="7:1", pos="7,0!"]
  8 [color=red, label="8:1", pos="8,0!"]
  9 [color=green, label="9:1", pos="9,0!"]
  10 [color=green, label="10", pos="0,1!"]
  11 [color=green, label="11", pos="1,1!"]
  12 [color=green, label="12", pos="4,1!"]
  13 [color=green, label="13", pos="5,1!"]
  14 [color=red, label="14", pos="6,1!"]
  15 [color=green, label="15:1", pos="-0.5,3!"]
  17 [color=red, label="17:1", pos="-5,2.5!"]
  19 [color=red, label="19:1", pos="-4,2.5!"]
  21 [color=red, label="21:1", pos="-3,2.5!"]
  23 [color=red, label="23:1", pos="-2,2.5!"]
  40 [color=green, label="40:1", pos="-2.5,5.5!"]
  41 [color=red, label="41:1", pos="-1,5.5!"]
  42 [color=green, label="42", pos="2,6.5!"]
  43 [

In [12]:
# Nomenclature taken from paper
E_CNT = 12
A_CNT = 9
OUT_CNT = 1 # always has to be zero but just to make it be clean

left_cnot_pos = []
right_cnot_pos = []
mid_toffoli_pos = 0

print("------------------------------------ Initial circuit creation ------------------------------------")

qcirc = QCirc(E_CNT, A_CNT, OUT_CNT)
qcirc.add_toffoli_gates_IS([0,1,2,3,4,5,6,7], E_CNT+0)
qcirc.add_not_gates_IS([6,7,11])
left_cnot_pos.append(len(qcirc.init_subgraphs)-1)
qcirc.add_toffoli_gates_IS([4,5,10,11], E_CNT+3)
qcirc.add_toffoli_gates_IS([2,3,4,5,9,11], E_CNT+6)
qcirc.add_not_gates_IS([4,5,10])
left_cnot_pos.append(len(qcirc.init_subgraphs)-1)
qcirc.add_toffoli_gates_IS([2,3,9,10], E_CNT+2)
qcirc.add_toffoli_gates_IS([0,1,2,3,8,10], E_CNT+5)
qcirc.add_not_gates_IS([2,3,9])
left_cnot_pos.append(len(qcirc.init_subgraphs)-1)
qcirc.add_toffoli_gates_IS([0,1,8,9], E_CNT+1)
qcirc.add_not_gates_IS([0,1])
left_cnot_pos.append(len(qcirc.init_subgraphs)-1)
qcirc.add_toffoli_gates_IS([0,1,2,3,4,5,6,7], E_CNT+0)
qcirc.add_toffoli_gates_IS([6,7,8,11], E_CNT+4)
qcirc.add_toffoli_gates_IS([4,5,6,7,8,10], E_CNT+7)
qcirc.add_not_gates_IS([6,7,8,11])
left_cnot_pos.append(len(qcirc.init_subgraphs)-1)
qcirc.add_toffoli_gates_IS([4,5,10,11], E_CNT+3)
qcirc.add_toffoli_gates_IS([2,3,4,5,9,11], E_CNT+6)
qcirc.add_not_gates_IS([4,5,10])
left_cnot_pos.append(len(qcirc.init_subgraphs)-1)
qcirc.add_toffoli_gates_IS([2,3,9,10], E_CNT+2)
qcirc.add_toffoli_gates_IS([0,1,2,3,8,10], E_CNT+5)
qcirc.add_not_gates_IS([9])
left_cnot_pos.append(len(qcirc.init_subgraphs)-1)
qcirc.add_toffoli_gates_IS([0,1,8,9], E_CNT+1)
qcirc.add_not_gates_IS([0,1])
left_cnot_pos.append(len(qcirc.init_subgraphs)-1)
qcirc.add_toffoli_gates_IS([6,7,8,11], E_CNT+4)
qcirc.add_not_gates_IS([9])
left_cnot_pos.append(len(qcirc.init_subgraphs)-1)
qcirc.add_toffoli_gates_IS([4,5,6,7,8,10], E_CNT+7)
qcirc.add_toffoli_gates_IS([0,1,6,7,9,11], E_CNT+8)
qcirc.add_not_gates_IS([0,1,6,7,9,11])
left_cnot_pos.append(len(qcirc.init_subgraphs)-1)
qcirc.add_toffoli_gates_IS([0,1,6,7,9,11], E_CNT+8)

mid_toffoli_ctrls = [0,E_CNT+0, E_CNT+1, E_CNT+2, E_CNT+3, E_CNT+4, E_CNT+5, E_CNT+6, E_CNT+7, E_CNT+8]
mid_toffoli_main_ctrl = E_CNT+A_CNT+0
qcirc.add_toffoli_gates_IS(mid_toffoli_ctrls, mid_toffoli_main_ctrl, False)
mid_toffoli_pos = len(qcirc.init_subgraphs)-2

qcirc.add_toffoli_gates_IS([0,1,6,7,9,11], E_CNT+8)
qcirc.add_not_gates_IS([0,1,6,7,9,11])
right_cnot_pos.append(len(qcirc.init_subgraphs)-1)
qcirc.add_toffoli_gates_IS([0,1,6,7,9,11], E_CNT+8)
qcirc.add_not_gates_IS([0,1,9])
right_cnot_pos.append(len(qcirc.init_subgraphs)-1)
qcirc.add_toffoli_gates_IS([4,5,6,7,8,10], E_CNT+7)
qcirc.add_toffoli_gates_IS([0,1,8,9], E_CNT+1)
qcirc.add_toffoli_gates_IS([6,7,8,11], E_CNT+4)
qcirc.add_not_gates_IS([4,5,6,7,9])
right_cnot_pos.append(len(qcirc.init_subgraphs)-1)
qcirc.add_toffoli_gates_IS([2,3,9,10], E_CNT+2)
qcirc.add_toffoli_gates_IS([0,1,2,3,8,10], E_CNT+5)
qcirc.add_not_gates_IS([8,10])
right_cnot_pos.append(len(qcirc.init_subgraphs)-1)
qcirc.add_toffoli_gates_IS([4,5,10,11], E_CNT+3)
qcirc.add_toffoli_gates_IS([2,3,4,5,9,11], E_CNT+6)
qcirc.add_not_gates_IS([11])
right_cnot_pos.append(len(qcirc.init_subgraphs)-1)
qcirc.add_toffoli_gates_IS([0,1,2,3,4,5,6,7], E_CNT+0)
qcirc.add_not_gates_IS([0,1,2,3])
right_cnot_pos.append(len(qcirc.init_subgraphs)-1)
qcirc.add_toffoli_gates_IS([6,7,8,11], E_CNT+4)
qcirc.add_toffoli_gates_IS([4,5,6,7,8,10], E_CNT+7)
qcirc.add_not_gates_IS([4,5,6,7])
right_cnot_pos.append(len(qcirc.init_subgraphs)-1)
qcirc.add_toffoli_gates_IS([0,1,8,9], E_CNT+1)
qcirc.add_not_gates_IS([9])
right_cnot_pos.append(len(qcirc.init_subgraphs)-1)
qcirc.add_toffoli_gates_IS([2,3,9,10], E_CNT+2)
qcirc.add_toffoli_gates_IS([0,1,2,3,8,10], E_CNT+5)
qcirc.add_not_gates_IS([10])
right_cnot_pos.append(len(qcirc.init_subgraphs)-1)
qcirc.add_toffoli_gates_IS([4,5,10,11], E_CNT+3)
qcirc.add_toffoli_gates_IS([2,3,4,5,9,11], E_CNT+6)
qcirc.add_not_gates_IS([11])
right_cnot_pos.append(len(qcirc.init_subgraphs)-1)
qcirc.add_toffoli_gates_IS([0,1,2,3,4,5,6,7], E_CNT+0)

qcirc.apply_diffusion_operator_IS()

qcirc.non_cliffs = [qcirc.get_merged_IS()]
qcirc.substitute_star_placeholder()
qcirc.revert_master_phases(qcirc.non_cliffs[0])

# somehow quizx doesn't like 5pi/3 phase, but 1pi/3 is fine
for v in qcirc.non_cliffs[0].vertices():
    if qcirc.non_cliffs[0].phase(v) == star_phase:
        qcirc.non_cliffs[0].set_phase(v, Fraction(1, 3))

qcirc.apply_effect_scalar_invariant(qcirc.non_cliffs[0], "/"*E_CNT + "1"*A_CNT + "-"*OUT_CNT)

nice_draw(qcirc.non_cliffs[0])

dot_string = to_dot(qcirc.non_cliffs[0]).rstrip()
print(dot_string)
with open("python_generated_dot_4eloop.txt", "w") as file:
    file.write(dot_string)

------------------------------------ Initial circuit creation ------------------------------------
	sqrt(2)^26 ≈ (8192.000000000015+0j)


graph {
  0 [color=green, label="0"]
  1 [color=green, label="1", pos="1,0!"]
  2 [color=green, label="2", pos="2,0!"]
  3 [color=green, label="3", pos="3,0!"]
  4 [color=green, label="4", pos="4,0!"]
  5 [color=green, label="5", pos="5,0!"]
  6 [color=green, label="6", pos="6,0!"]
  7 [color=green, label="7", pos="7,0!"]
  8 [color=green, label="8", pos="8,0!"]
  9 [color=green, label="9", pos="9,0!"]
  10 [color=green, label="10", pos="10,0!"]
  11 [color=green, label="11", pos="11,0!"]
  12 [color=red, label="12:1", pos="12,0!"]
  13 [color=red, label="13:1", pos="13,0!"]
  14 [color=red, label="14:1", pos="14,0!"]
  15 [color=red, label="15:1", pos="15,0!"]
  16 [color=red, label="16:1", pos="16,0!"]
  17 [color=red, label="17:1", pos="17,0!"]
  18 [color=red, label="18:1", pos="18,0!"]
  19 [color=red, label="19:1", pos="19,0!"]
  20 [color=red, label="20:1", pos="20,0!"]
  21 [color=green, label="21:1", pos="21,0!"]
  22 [color=green, label="22", pos="0,1!"]
  23 [color=green, la